In [1]:
from copy import deepcopy
from functools import cache
from os import listdir
import json
import pickle
from datetime import datetime

from scipy.special import softmax
import numpy as np
import math
from itertools import combinations

from typing import Any, List, Tuple
import time

import torch
import dataclasses


# |Set root for GPUDrive import
import os
import sys
from pathlib import Path

from traitlets import default

# Set working directory to the base directory 'gpudrive'
working_dir = Path.cwd()
while working_dir.name != 'gpudrive-CoDec':
    working_dir = working_dir.parent
    if working_dir == Path.home():
        raise FileNotFoundError("Base directory 'gpudrive' not found")
os.chdir(working_dir)
sys.path.append(str(working_dir))


# |GPUDrive imports
from gpudrive.utils.config import load_config
from examples.CoDec_Research.code.simulation.construal_main import generate_baseline_data, generate_selected_construal_traj, \
                                                                    get_constral_heurisrtic_values, generate_all_construal_trajnval
from examples.CoDec_Research.code.gpuDrive_utils import get_gpuDrive_vars
from examples.CoDec_Research.code.analysis.evaluate_construal_actions import evaluate_construals, get_best_construals_likelihood



RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

2025-05-06 01:49:53.233921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746510593.250340  196998 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746510593.255329  196998 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746510593.271161  196998 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746510593.271190  196998 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746510593.271191  196998 computation_placer.cc:177] computation placer alr

RuntimeError: module compiled against ABI version 0x1000009 but this version of numpy is 0x2000000

In [2]:

### Parameters for Inference ###
construal_selection_logic = "heuristic_1" # Set Construal Selection Logic. 
                                          # Options: "log_likelihood", "heuristic_1", "heuristic_2", "heuristic_3"
heuristic_params = {"dist_ego": 0.5} # Parameters for weighing the heuristics

### Specify Environment Configuration ###

# |Location to store (and find pre-computed) simulation results
simulation_results_path = "examples/CoDec_Research/results/simulation_results/"

# |Location to store simulation results
out_dir = "examples/CoDec_Research/results/simulation_results/"

# |Model Config (on which model was trained)
training_config = load_config("examples/experimental/config/reliable_agents_params")

# |Set scenario path
dataset_path = 'data/processed/construal'

# |Set simulator config
max_agents = training_config.max_controlled_agents   # Get total vehicle count
num_parallel_envs = 2
total_envs = 4
device = "cpu" # cpu just because we're in a notebook
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# |Set construal config
construal_size = 1
observed_agents_count = max_agents - 1      # Agents observed except self (used for vector sizes)
sample_size = 1                             # Number of samples to calculate expected utility of a construal

# |Other changes to variables
training_config.max_controlled_agents = 1    # Control only the first vehicle in the environment
total_envs = min(total_envs, len(listdir(dataset_path)))


### Instantiate Variables ###

env_config, train_loader, env, env_multi_agent, sim_agent = get_gpuDrive_vars(
                                                                                training_config = training_config,
                                                                                device = device,
                                                                                num_parallel_envs = num_parallel_envs,
                                                                                dataset_path = dataset_path,
                                                                                max_agents = max_agents,
                                                                                total_envs = total_envs,
                                                                                sim_agent_path= "daphne-cornelisse/policy_S10_000_02_27",
                                                                            )


In [3]:

### Select Construals for Baseline Data ###
scene_constr_dict = None
if construal_selection_logic == "heuristic_1":
    # |Select construals based on heuristic 1 (distance from ego)

    #2# |Generate Construal Execution Values through simulator sampling
    default_values = None

    #2# |Check if saved data is available
    simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]
    for srFile in simulation_results_files:
        if "construal_vals" in srFile:
            with open(srFile, 'rb') as opn_file:
                default_values = pickle.load(opn_file)
        else:
            continue

    if default_values is None:
        default_values, traj_obs, ground_truth = generate_all_construal_trajnval(out_dir=out_dir,
                                                                                    sim_agent=sim_agent,
                                                                                    observed_agents_count=observed_agents_count,
                                                                                    construal_size=construal_size,
                                                                                    num_parallel_envs=num_parallel_envs,
                                                                                    max_agents=max_agents,
                                                                                    sample_size=sample_size,
                                                                                    device=device,
                                                                                    train_loader=train_loader,
                                                                                    env=env,
                                                                                    env_multi_agent=env_multi_agent,
                                                                                    generate_animations=False)

    #2# |Generate Construal Heuristic Values (Heuristic 1: Distance from ego)
    heuristic_values = get_constral_heurisrtic_values(env, train_loader, default_values, heuristic=1, heuristic_params=heuristic_params)

    #2# |Sample from construal values
    def sample_construals(heuristic_values: dict, sample_count: int) -> dict:
        """
        Sample construals based on heuristic values.
        """
        sampled_construals = {}
        for scene_name, construal_info in heuristic_values.items():
            constr_indices, constr_values = zip(*construal_info.items())
            constr_values_softmax = torch.nn.functional.softmax(torch.tensor(constr_values), dim=0)
            sampled_indices = torch.multinomial(constr_values_softmax, num_samples=sample_count, \
                                                               replacement=False).tolist()
            sampled_construals[scene_name] = {constr_indices[i]: constr_values[i] for i in sampled_indices}

        return sampled_construals
    
    scene_constr_dict = sample_construals(heuristic_values, sample_count=2)




Processing Waymo batches: 100%|██████████| 2/2 [00:00<00:00,  7.99it/s]


In [4]:
heuristic_values

{'tfrecord-00101-of-01000_71': {(0,): np.float64(1.0),
  (1,): np.float64(0.9548024362736818),
  (5,): np.float64(0.629016666487872),
  (9,): np.float64(0.5742991115584066),
  (10,): np.float64(0.9042478755727865),
  (42,): np.float64(0.613054930528357),
  (43,): np.float64(0.5385952434420198),
  (0, 1, 5, 9, 10, 42, 43): np.float64(0.7448594662661605)},
 'tfrecord-00117-of-01000_240': {(0,): np.float64(1.0),
  (1,): np.float64(0.7215077800397827),
  (2,): np.float64(0.5381755943648112),
  (3,): np.float64(0.6078264890552836),
  (4,): np.float64(0.9401724415680937),
  (5,): np.float64(0.700456969135634),
  (7,): np.float64(0.9385660133622136),
  (8,): np.float64(0.6320838417348973),
  (9,): np.float64(0.744795752946394),
  (10,): np.float64(0.6767799318279193),
  (11,): np.float64(0.7732606874164518),
  (12,): np.float64(0.827249699708235),
  (13,): np.float64(0.7074301356948892),
  (14,): np.float64(0.7919905805414902),
  (15,): np.float64(0.8085967087013745),
  (16,): np.float64(0.57

In [5]:
### Generate Trajectories For Selected Construals (Baseline Data on Which to Perform Inference) ###
state_action_pairs = None

# |Check if saved data is available
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]
for srFile in simulation_results_files:
    if "baseline_state_action_pairs" not in srFile:
        continue
    with open(srFile, 'rb') as opn_file:
        state_action_pairs = pickle.load(opn_file)

if state_action_pairs is None:
    state_action_pairs = generate_baseline_data(out_dir=out_dir,
                                                sim_agent=sim_agent,
                                                num_parallel_envs=num_parallel_envs,
                                                max_agents=max_agents,
                                                sample_size=1,
                                                device=device,
                                                train_loader=train_loader,
                                                env=env,
                                                env_multi_agent=env_multi_agent,
                                                observed_agents_count=observed_agents_count,
                                                construal_size=construal_size,
                                                selected_construals=scene_constr_dict,
                                                generate_animations=False)


In [6]:
### Compute Construal Log Likelihoods ###
construal_action_likelihoods = None

# |Check if saved data is available
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]
for srFile in simulation_results_files:
    if "log_likelihood_measures" not in srFile:
        continue
    with open(srFile, 'rb') as opn_file:
        construal_action_likelihoods = pickle.load(opn_file)

if construal_action_likelihoods is None:
    construal_action_likelihoods = evaluate_construals(state_action_pairs, construal_size, sim_agent, out_dir, device=device)

# |Clear memory for large variable, once it has served its purpose
# del state_action_pairs




Processing Scene:  tfrecord-00101-of-01000_71
(42,) (0,)
Processing baseline construal (42,) against construal (0,), sample 0
(42,) (1,)
Processing baseline construal (42,) against construal (1,), sample 0
(42,) (5,)
Processing baseline construal (42,) against construal (5,), sample 0
(42,) (9,)
Processing baseline construal (42,) against construal (9,), sample 0
(42,) (10,)
Processing baseline construal (42,) against construal (10,), sample 0
(42,) (42,)
Processing baseline construal (42,) against construal (42,), sample 0
(42,) (43,)
Processing baseline construal (42,) against construal (43,), sample 0
(42,) (0, 1, 5, 9, 10, 42, 43)
Processing baseline construal (42,) against construal (0, 1, 5, 9, 10, 42, 43), sample 0
(0, 1, 5, 9, 10, 42, 43) (0,)
Processing baseline construal (0, 1, 5, 9, 10, 42, 43) against construal (0,), sample 0
(0, 1, 5, 9, 10, 42, 43) (1,)
Processing baseline construal (0, 1, 5, 9, 10, 42, 43) against construal (1,), sample 0
(0, 1, 5, 9, 10, 42, 43) (5,)
Pr

In [7]:

### Sanity Check ###
scene_constr_dict = None
scene_constr_diff_dict = None

# |Check if saved data is available
simulation_results_files = [simulation_results_path+fl_name for fl_name in listdir(simulation_results_path)]
for srFile in simulation_results_files:
    if "highest_construal_dict_log_likelihood_diff" in srFile:
        with open(srFile, 'rb') as opn_file:
            scene_constr_diff_dict = pickle.load(opn_file)
    if "highest_construal_dict_log_likelihood" in srFile:
        with open(srFile, 'rb') as opn_file:
            scene_constr_dict = pickle.load(opn_file)

if scene_constr_dict is None:
    scene_constr_dict = get_best_construals_likelihood(construal_action_likelihoods, out_dir)
if scene_constr_diff_dict is None:
    scene_constr_diff_dict = get_best_construals_likelihood(construal_action_likelihoods, out_dir, likelihood_key="log_likelihood_diff")


Highest scene construals dict saved to:  examples/CoDec_Research/results/simulation_results/highest_construal_dict_log_likelihood_2025-05-06 01:50:43.012298.pickle
Highest scene construals dict saved to:  examples/CoDec_Research/results/simulation_results/highest_construal_dict_log_likelihood_diff_2025-05-06 01:50:43.013299.pickle


In [8]:
scene_constr_dict

{'tfrecord-00101-of-01000_71': {(42,): ((0, 1, 5, 9, 10, 42, 43),
   43.80292510986328),
  (0, 1, 5, 9, 10, 42, 43): ((0, 1, 5, 9, 10, 42, 43), 44.92628479003906)},
 'tfrecord-00117-of-01000_240': {(5,): ((0,), 95.89288330078125),
  (16,): ((5,), 98.80965423583984)},
 'tfrecord-00143-of-01000_179': {(0,): ((0,), 89.8375015258789),
  (6,): ((9,), 87.57784271240234)},
 'tfrecord-00144-of-01000_130': {(0,): ((7,), 49.09541702270508),
  (59,): ((0,), 45.58967590332031)}}

In [ ]:
for scene_name_, scene_info_ in construal_action_likelihoods.items():
    print(f"Scene: {scene_name_}")
    for base_construal_name_, base_construal_info_ in scene_info_.items():
        print_dict = {}
        for test_construal_name_, test_construal_info_ in base_construal_info_.items():
            for sample_num_, sample_info_ in test_construal_info_.items():
                print_dict.update({(base_construal_name_, test_construal_name_, sample_num_): abs(sample_info_['log_likelihood_diff'])})
                # print_dict.update({(base_construal_name_, test_construal_name_, sample_num_): sample_info_['log_likelihood']})
        print(dict(sorted(print_dict.items(), key=lambda item: item[1])))

Scene: tfrecord-00101-of-01000_71
Base Construal: (42,), Test Construal: (0,), Sample: 0
Log Likelihood Diff: -5.185614099900704e-06
Base Construal: (42,), Test Construal: (1,), Sample: 0
Log Likelihood Diff: 11.97215461730957
Base Construal: (42,), Test Construal: (5,), Sample: 0
Log Likelihood Diff: 4.085542678833008
Base Construal: (42,), Test Construal: (9,), Sample: 0
Log Likelihood Diff: -0.14077667891979218
Base Construal: (42,), Test Construal: (10,), Sample: 0
Log Likelihood Diff: 0.06767607480287552
Base Construal: (42,), Test Construal: (42,), Sample: 0
Log Likelihood Diff: -5.185614099900704e-06
Base Construal: (42,), Test Construal: (43,), Sample: 0
Log Likelihood Diff: -5.185614099900704e-06
Base Construal: (42,), Test Construal: (0, 1, 5, 9, 10, 42, 43), Sample: 0
Log Likelihood Diff: 13.686068534851074
Base Construal: (0, 1, 5, 9, 10, 42, 43), Test Construal: (0,), Sample: 0
Log Likelihood Diff: -10.172041893005371
Base Construal: (0, 1, 5, 9, 10, 42, 43), Test Construa

In [ ]:
### Construal Inference ###
for scene_name, scene_info in heuristic_values.items():
    base_construal = 0
    for construal_indx, construal_heur_value in scene_info.items():
        sample_num = 0
        p_a = construal_action_likelihoods[scene_name][base_construal][construal_indx][sample_num]['log_likelihood'].item()
        p_lambda = torch.exp(p_a)*construal_heur_value.item()